In [ ]:
import os

from srai.datasets import HouseSalesInKingCountyDataset

%load_ext dotenv
%dotenv

In [ ]:
hskc_dataset = HouseSalesInKingCountyDataset()
hf_token = os.getenv("HF_TOKEN")

In [ ]:
gdf = hskc_dataset.load(hf_token=hf_token)

In [ ]:
gdf.shape